In [65]:
import pandas
import sys
import re
import os

In [66]:
os.chdir("/home/afollette/jupyter_notebooks/pdmr_fastq_proccessing")

In [67]:
pdmrData = open("proccessedEnaSamplePdmr", 'r')
rawData = open("pdmrNeo4jExport.csv", 'r')
rawDf = pandas.read_csv(rawData, sep=',')
pdmrDf = pandas.read_csv(pdmrData, sep=',')

In [69]:
pdmrDf['sample_title'] = pdmrDf['Unnamed: 0'] + "-" + pdmrDf['Unnamed: 1']
rawDf['sample_title'] = rawDf['mod.sourcePdxId'] + "-" + rawDf['xS.sourceSampleId']

In [70]:
genderLookup = pandas.DataFrame([rawDf['sample_title'],rawDf['p.sex']]).transpose().set_index('sample_title')
pdmrDf.set_index('sample_title', inplace=True)

In [71]:
completedData = pdmrDf.join(genderLookup,on="sample_title").reset_index()
completedData.fillna('',inplace=True)
completedData['patient tumor type'] = completedData['patient tumor type'] + " neoplasm" 
completedData['sample_title'] = completedData['Unnamed: 0'].str.replace("-","~",n=1) + "~" + completedData['Unnamed: 1'].str.replace("-","~")

In [72]:
def f(x):
    if x=="Patient":
        return "Homo sapien"
    elif x=="xenograft":
        return "Homo sapien/Mus musculus" 

In [73]:
completedData["sample taxon name"] = completedData["sample origin"].apply(lambda x: f(x))

In [75]:
completedData.to_csv('completedSampledata',sep='\t')

In [76]:
fastqWes = open('pdmrWesList', 'r')
fastqRna = open('pdmrRnaSeqList', 'r')
fastqWesDf = pandas.read_csv(fastqWes)
fastqRnaDf = pandas.read_csv(fastqRna)
fastqWesDf.columns=["wesfilename"]
fastqRnaDf.columns=["rnafilename"]

In [77]:
fastqRnaDf['sample_title'] = fastqRnaDf['rnafilename'].apply(lambda x: re.findall('^.+\~v', x).pop()[:-2])
fastqRnaDf['sample_id'] = fastqRnaDf['rnafilename'].apply(lambda x: re.findall('\w+\~v', x).pop()[:-2])
fastqWesDf['sample_title'] = fastqWesDf['wesfilename'].apply(lambda x: re.findall('^.+\~v', x).pop()[:-2])
fastqWesDf['sample_id'] = fastqWesDf['wesfilename'].apply(lambda x: re.findall('\w+\~v', x).pop()[:-2])

In [78]:
completedData.set_index('sample_title', inplace=True)
fastqWesDf.set_index('sample_title', inplace=True)
fastqRnaDf.set_index('sample_title', inplace=True)

In [85]:
fastqWes = completedData.join(fastqWesDf, on='sample_title', how='left')


In [92]:
fastqCheck = fastqWes.join(fastqRnaDf, on='sample_title', how='left', lsuffix='wes', rsuffix='rna')

In [94]:
fastqCheck.to_csv('completedFastqCheck',sep='\t')